In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer
import pandas as pd
import matplotlib.pyplot as plt

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
import re

# GPU가 있으면 사용, 아닐경우 CPU사용 (GPU사용을 추천합니다)

In [2]:
tf.get_logger().setLevel('ERROR')

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)

# Data 읽어오기 및 전처리
result_text가 원본 text이며,  text_processed가 전처리된 text입니다.

result_text를 text_preprocessed로 바꾸는 과정을 "텍스트 전처리" 라고 부르겠습니다.
텍스트 전처리를 수행한이유는 Bert 전처리에서 설명되어있습니다.

In [29]:
csv = pd.read_csv('./helico_preprocessed.csv')
csv = csv.dropna(axis=0)

In [30]:
csv.head()

,ID,SM_DATE,EXEC_TIME,처방코드,result_text,h_pyl_positive,text_processed
0,00A020CFDF2D,2019-04-11 00:00:00,2019-04-11 13:46:56,BP1A151A,▣ 결론 및 진단\r\n\r\n(507011)\r\n\r\n ...,1,507011 THE UPDATED SYDNEY SYSTEM Site H. pylo...
1,00A0347B006B,2020-08-07 00:00:00,2020-08-07 15:51:42,BP1A151A,▣ 결론 및 진단\r\n\r\n(219350)\r\n\r\n ...,0,219350 THE UPDATED SYDNEY SYSTEM Site H. pylo...
2,00A091037DE1,2019-01-25 00:00:00,2019-01-25 14:36:40,BP1A151A,"▣ 결론 및 진단\r\n\r\n(507011)\r\n\r\n1. Stomach, #...",0,"507011 1. Stomach, #1x1 : GC of low body, bio..."
3,00A0A50F463F,2018-12-21 00:00:00,2018-12-21 15:59:49,BP1A151A,"▣ 결론 및 진단\r\n\r\n(PP4970)\r\n\r\n Stomach, #1...",0,"PP4970 Stomach, #1x1 : Anterior wall of low b..."
4,00A0C41326F7,2019-05-17 00:00:00,2019-05-17 15:46:34,BP1A151A,"▣ 결론 및 진단\r\n\r\n(298059)\r\n\r\n Stomach, #1...",0,"298059 Stomach, #1x1 : fundus, biopsy : . Fun..."


# 텍스트 전처리 전

In [31]:
print(csv['result_text'][0])

▣ 결론 및 진단

(507011)

                              THE UPDATED SYDNEY SYSTEM
Site            H. pylori       Neutrophil    Mononuclear   Atrophy     Intestinal
                colonization     activity          cells                 metaplasia
------------------------------------------------------------------------------------
1.#1x1 : antrum    mild         mild           moderate      not applicable  absent    
------------------------------------------------------------------------------------
2.#2x1 : antrum    mild         mild           moderate      not applicable  absent

1. Stomach, #1x1 : LC of distal antrum, biopsy :

  . H. pylori-chronic gastritis, active, with erosion

2. Stomach, #2x1 : LC of prepyloric antrum, biopsy :

  . H. pylori-chronic gastritis, active, with erosion






# 텍스트 전처리 후

In [32]:
print(csv['text_processed'][0])

 507011 THE UPDATED SYDNEY SYSTEM Site H. pylori Neutrophil Mononuclear Atrophy Intestinal colonization activity cells metaplasia 1.#1x1 : antrum mild mild moderate not applicable absent 2.#2x1 : antrum mild mild moderate not applicable absent 1. Stomach, #1x1 : LC of distal antrum, biopsy : . H. pylori chronic gastritis, active, with erosion 2. Stomach, #2x1 : LC of prepyloric antrum, biopsy : . H. pylori chronic gastritis, active, with erosion 


In [24]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(csv, test_size=0.2, random_state=1004, stratify=csv['h_pyl_positive'])

# Bert 전처리 모델 읽어오기

In [12]:
bert_model_name = 'experts_wiki_books' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

BERT model selected           : https://tfhub.dev/google/experts/bert/wiki_books/2
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


# 텍스트 전처리의 이유
## 텍스트 전처리 이전

THE UPDATED SYDNEY SYSTEM를 삭제한 이유는 다음과같습니다. 텍스트 전처리 이전의 text인 'result_text'를 넣었을때에는 bert preprocess model의 output에서 input_word_ids에 1027이라는 숫자가 많이보입니다. 즉 의미없는 데이터(분류하는데 크게 기여하지않으리라고 예상이되는)가 많이 존재합니다.

In [25]:
bert_preprocess_model([train['result_text'][0]])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,   100,  1455, 30010, 30022, 29994, 30011, 30021,   100,
          1464, 30019, 30021, 29993, 30006, 30021,  1006,  2753, 19841,
         14526,  1007,  1996,  7172,  3994,  2291,  1027,  1027,  1027,
          1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,
          1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,
          1027,  1027,  1027,  1027,  1027,  102

In [26]:
bert_preprocess_model([train['result_text'][1]])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,   100,  1455, 30010, 30022, 29994, 30011, 30021,   100,
          1464, 30019, 30021, 29993, 30006, 30021,  1006, 20636, 19481,
          2692,  1007,  1996,  7172,  3994,  2291,  1027,  1027,  1027,
          1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,
          1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,  1027,
          1027,  1027,  1027,  1027,  1027,  102

## 텍스트 전처리 후
반대로, THE UPDATED SYDNEY SYSTEM를 제거한 이후 bert_preprocess_model에 넣어준 결과는 다음과같습니다. 텍스트 전처리 이전과는 다르게 많은부분을 차지하고있던 1027이 삭제된것을 확인할 수 있습니다.

In [27]:
bert_preprocess_model([train['text_processed'][0]])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  2753, 19841, 14526,  1996,  7172,  3994,  2291,  2609,
          1044,  1012,  1052,  8516, 10050, 11265,  4904, 18981, 19466,
         18847, 11231, 14321,  2906,  2012, 18981, 10536, 20014, 19126,
         18962,  4023,  4442, 18804, 24759, 15396,  1015,  1012,  1001,
          1015,  2595,  2487,  1024, 14405,  6824, 10256, 10256,  8777,
          2025, 12711,  9962,  1016,  1012,  100

In [28]:
bert_preprocess_model([train['text_processed'][1]])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101, 20636, 19481,  2692,  1996,  7172,  3994,  2291,  2609,
          1044,  1012,  1052,  8516, 10050, 11265,  4904, 18981, 19466,
         18847, 11231, 14321,  2906,  2012, 18981, 10536, 20014, 19126,
         18962,  4023,  4442, 18804, 24759, 15396,  1015,  1012,  1001,
          1015,  2595,  2487,  1024, 14405,  6824,  9962,  9962, 10256,
          2025, 12711,  4417,  1016,  1012,  100

# 모델 build 및 학습.

In [17]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [19]:
classifier_model = build_classifier_model()
# bert_raw_result = classifier_model(tf.constant(test_text))
# print(tf.sigmoid(bert_raw_result))

# metrics = [tf.metrics.BinaryAccuracy(), tf.metrics.AUC(), tf.metrics.Recall()]
batch_size = 32
epochs = 5
steps_per_epoch = len(train) // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer,
                         loss='binary_crossentropy',
                         metrics=['accuracy'])

# 텍스트 전처리 후의 데이터를 사용한 모델

In [20]:
history = classifier_model.fit(train['text_processed'], train['h_pyl_positive'],
                               batch_size=batch_size,
                               epochs=2)

Epoch 1/2
3912/3912 [==============================] - 1981s 502ms/step - loss: 0.1428 - accuracy: 0.9275
Epoch 2/2
1857/3912 [=============>................] - ETA: 17:19 - loss: 0.0151 - accuracy: 0.9972

KeyboardInterrupt: 

In [ ]:
classifier_model.evaluate(val['text_processed'], val['h_pyl_positive'])

# 텍스트 전처리 이전의 데이터를 사용한 모델

In [36]:
classifier_model = build_classifier_model()
# bert_raw_result = classifier_model(tf.constant(test_text))
# print(tf.sigmoid(bert_raw_result))

# metrics = [tf.metrics.BinaryAccuracy(), tf.metrics.AUC(), tf.metrics.Recall()]
batch_size = 32
epochs = 2
steps_per_epoch = len(train) // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer,
                         loss='binary_crossentropy',
                         metrics=['accuracy'])

In [37]:
history = classifier_model.fit(train['result_text'], train['h_pyl_positive'],
                               batch_size=batch_size,
                               epochs=epochs)

Epoch 1/2
311/311 [==============================] - 176s 506ms/step - loss: 0.6514 - accuracy: 0.6466
Epoch 2/2
311/311 [==============================] - 159s 512ms/step - loss: 0.6282 - accuracy: 0.6505
